# Can a single global model learn from multiple energy-demand domains using a domain indicator?

In [2]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

plt.style.use("seaborn-v0_8-darkgrid")

In [11]:
df_delhi = pd.read_csv("../results/Delhi_Hourly_train.csv", parse_dates=["ds"])
df_delhi["region_id"] = 0           # 0 = Delhi (City-Level)
df_delhi

,ds,y,temp,dwpt,rhum,wdir,wspd,pres,hour,weekday,month,is_weekend,hour_sin,hour_cos,lag_1,lag_24,lag_168,rolling_mean_3,rolling_mean_24,region_id
0,2021-01-08 00:00:00,4697.857500,13.0,11.60,91.0,145.0,2.7,1014.85,0,4,1,0,0.000000,1.000000,6723.608333,6557.406667,1975.541667,6566.744722,5909.383785,0
1,2021-01-08 01:00:00,4430.386667,13.0,12.10,94.0,0.0,0.0,1014.00,1,4,1,0,0.258819,0.965926,4697.857500,6331.355833,1805.590000,6048.358056,5831.902569,0
2,2021-01-08 02:00:00,4238.806667,12.6,12.15,97.0,0.0,0.0,1014.40,2,4,1,0,0.500000,0.866025,4430.386667,6075.757500,1665.914167,5283.950833,5752.695521,0
3,2021-01-08 03:00:00,4055.965833,12.1,12.10,100.0,0.0,0.0,1013.90,3,4,1,0,0.707107,0.707107,4238.806667,5798.060000,1607.331667,4455.683611,5676.155903,0
4,2021-01-08 04:00:00,3894.686667,12.0,12.00,100.0,0.0,0.0,1013.00,4,4,1,0,0.866025,0.500000,4055.965833,5545.454167,1676.528333,4241.719722,5603.568646,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27528,2024-02-29 00:00:00,2081.147500,18.0,10.35,61.0,290.0,7.6,1015.40,0,3,2,0,0.000000,1.000000,2720.321667,2053.207500,2091.685833,2785.581944,3036.565938,0
27529,2024-02-29 01:00:00,1917.576667,17.5,10.00,61.5,280.0,6.5,1014.50,1,3,2,0,0.258819,0.965926,2081.147500,1885.863333,1930.974167,2507.263611,3037.730104,0
27530,2024-02-29 02:00:00,1826.179167,16.0,10.45,70.0,255.0,6.2,1014.00,2,3,2,0,0.500000,0.866025,1917.576667,1814.252500,1843.295833,2239.681944,3039.051493,0
27531,2024-02-29 03:00:00,1791.240833,14.5,11.00,79.5,250.0,7.3,1014.00,3,3,2,0,0.707107,0.707107,1826.179167,1781.202500,1817.735833,1941.634444,3039.548438,0


In [14]:
df_residential = pd.read_csv("../results/Residential_processed.csv", parse_dates=["ds"])
df_residential["region_id"] = 1         # 1 = Residential
df_residential

,ds,y,hour,weekday,month,is_weekend,hour_sin,hour_cos,lag_1,lag_24,lag_168,rolling_mean_3,rolling_mean_24,region_id
0,2016-01-08 06:00:00+00:00,0.331117,6,4,1,0,1.000000,6.123234e-17,0.459283,0.380783,0.527133,0.536206,0.825481,1
1,2016-01-08 07:00:00+00:00,0.351633,7,4,1,0,0.965926,-2.588190e-01,0.331117,0.291400,0.463150,0.423322,0.823411,1
2,2016-01-08 08:00:00+00:00,0.382350,8,4,1,0,0.866025,-5.000000e-01,0.351633,0.255817,0.497000,0.380678,0.825921,1
3,2016-01-08 09:00:00+00:00,0.373350,9,4,1,0,0.707107,-7.071068e-01,0.382350,0.376933,0.498317,0.355033,0.831193,1
4,2016-01-08 10:00:00+00:00,0.330533,10,4,1,0,0.500000,-8.660254e-01,0.373350,0.281533,0.445383,0.369111,0.831044,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2711,2016-04-30 05:00:00+00:00,0.412917,5,5,4,1,0.965926,2.588190e-01,1.924700,0.633900,3.620583,2.172489,1.809899,1
2712,2016-04-30 06:00:00+00:00,1.417400,6,5,4,1,1.000000,6.123234e-17,0.412917,1.556333,0.498550,1.871144,1.800692,1
2713,2016-04-30 07:00:00+00:00,0.895467,7,5,4,1,0.965926,-2.588190e-01,1.417400,0.710667,0.520250,1.251672,1.794903,1
2714,2016-04-30 08:00:00+00:00,0.641883,8,5,4,1,0.866025,-5.000000e-01,0.895467,0.535050,0.527917,0.908594,1.802603,1


In [16]:
weather_cols = ["temp", "dwpt", "rhum", "wdir", "wspd", "pres"]
for col in weather_cols:
    if col not in df_residential.columns:
        df_residential[col] = 0.0

,ds,y,hour,weekday,month,is_weekend,hour_sin,hour_cos,lag_1,lag_24,lag_168,rolling_mean_3,rolling_mean_24,region_id,temp,dwpt,rhum,wdir,wspd,pres
0,2016-01-08 06:00:00+00:00,0.331117,6,4,1,0,1.000000,6.123234e-17,0.459283,0.380783,0.527133,0.536206,0.825481,1,0.0,0.0,0.0,0.0,0.0,0.0
1,2016-01-08 07:00:00+00:00,0.351633,7,4,1,0,0.965926,-2.588190e-01,0.331117,0.291400,0.463150,0.423322,0.823411,1,0.0,0.0,0.0,0.0,0.0,0.0
2,2016-01-08 08:00:00+00:00,0.382350,8,4,1,0,0.866025,-5.000000e-01,0.351633,0.255817,0.497000,0.380678,0.825921,1,0.0,0.0,0.0,0.0,0.0,0.0
3,2016-01-08 09:00:00+00:00,0.373350,9,4,1,0,0.707107,-7.071068e-01,0.382350,0.376933,0.498317,0.355033,0.831193,1,0.0,0.0,0.0,0.0,0.0,0.0
4,2016-01-08 10:00:00+00:00,0.330533,10,4,1,0,0.500000,-8.660254e-01,0.373350,0.281533,0.445383,0.369111,0.831044,1,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2711,2016-04-30 05:00:00+00:00,0.412917,5,5,4,1,0.965926,2.588190e-01,1.924700,0.633900,3.620583,2.172489,1.809899,1,0.0,0.0,0.0,0.0,0.0,0.0
2712,2016-04-30 06:00:00+00:00,1.417400,6,5,4,1,1.000000,6.123234e-17,0.412917,1.556333,0.498550,1.871144,1.800692,1,0.0,0.0,0.0,0.0,0.0,0.0
2713,2016-04-30 07:00:00+00:00,0.895467,7,5,4,1,0.965926,-2.588190e-01,1.417400,0.710667,0.520250,1.251672,1.794903,1,0.0,0.0,0.0,0.0,0.0,0.0
2714,2016-04-30 08:00:00+00:00,0.641883,8,5,4,1,0.866025,-5.000000e-01,0.895467,0.535050,0.527917,0.908594,1.802603,1,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
global_features = [
    # Weather
    "temp", "dwpt", "rhum", "wdir", "wspd", "pres",

    # Time
    "hour", "weekday", "month", "is_weekend", "hour_sin", "hour_cos",

    # Lags and Rolls
    "lag_1", "lag_24", "lag_168", "rolling_mean_3", "rolling_mean_24",

    # Domain Indicator
    "region_id"
]

In [21]:
df_global = pd.concat([df_delhi, df_residential], axis=0).reset_index(drop=True)
df_global

,ds,y,temp,dwpt,rhum,wdir,wspd,pres,hour,weekday,month,is_weekend,hour_sin,hour_cos,lag_1,lag_24,lag_168,rolling_mean_3,rolling_mean_24,region_id
0,2021-01-08 00:00:00,4697.857500,13.0,11.60,91.0,145.0,2.7,1014.85,0,4,1,0,0.000000,1.000000e+00,6723.608333,6557.406667,1975.541667,6566.744722,5909.383785,0
1,2021-01-08 01:00:00,4430.386667,13.0,12.10,94.0,0.0,0.0,1014.00,1,4,1,0,0.258819,9.659258e-01,4697.857500,6331.355833,1805.590000,6048.358056,5831.902569,0
2,2021-01-08 02:00:00,4238.806667,12.6,12.15,97.0,0.0,0.0,1014.40,2,4,1,0,0.500000,8.660254e-01,4430.386667,6075.757500,1665.914167,5283.950833,5752.695521,0
3,2021-01-08 03:00:00,4055.965833,12.1,12.10,100.0,0.0,0.0,1013.90,3,4,1,0,0.707107,7.071068e-01,4238.806667,5798.060000,1607.331667,4455.683611,5676.155903,0
4,2021-01-08 04:00:00,3894.686667,12.0,12.00,100.0,0.0,0.0,1013.00,4,4,1,0,0.866025,5.000000e-01,4055.965833,5545.454167,1676.528333,4241.719722,5603.568646,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30244,2016-04-30 05:00:00+00:00,0.412917,0.0,0.00,0.0,0.0,0.0,0.00,5,5,4,1,0.965926,2.588190e-01,1.924700,0.633900,3.620583,2.172489,1.809899,1
30245,2016-04-30 06:00:00+00:00,1.417400,0.0,0.00,0.0,0.0,0.0,0.00,6,5,4,1,1.000000,6.123234e-17,0.412917,1.556333,0.498550,1.871144,1.800692,1
30246,2016-04-30 07:00:00+00:00,0.895467,0.0,0.00,0.0,0.0,0.0,0.00,7,5,4,1,0.965926,-2.588190e-01,1.417400,0.710667,0.520250,1.251672,1.794903,1
30247,2016-04-30 08:00:00+00:00,0.641883,0.0,0.00,0.0,0.0,0.0,0.00,8,5,4,1,0.866025,-5.000000e-01,0.895467,0.535050,0.527917,0.908594,1.802603,1


In [22]:
df_global.shape

(30249, 20)

In [23]:
split_idx = int(len(df_global)*0.8)

train_df = df_global.iloc[:split_idx]
test_df = df_global.iloc[split_idx:]

In [25]:
X_train = train_df[global_features]
X_test = test_df[global_features]

y_train = train_df["y"]
y_test = test_df["y"]

In [26]:
global_lgbm = LGBMRegressor(
    n_estimators=500,
    learning_rate=0.05,
    num_leaves=31,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
global_lgbm.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000826 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2748
[LightGBM] [Info] Number of data points in the train set: 24199, number of used features: 17
[LightGBM] [Info] Start training from score 3870.014802
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive ga

,boosting_type,'gbdt'
,num_leaves,31
,max_depth,6
,learning_rate,0.05
,n_estimators,500
,subsample_for_bin,200000
,objective,None
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


In [28]:
y_pred = global_lgbm.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Global Model -> MAE : {mae:.2f} and RMSE : {rmse:.2f}")

Global Model -> MAE : 722.30 and RMSE : 1011.33


In [35]:
test_df = test_df.copy()
test_df["y_pred"] = y_pred

for rid, name in [(0, 'Delhi'), (1, 'Residential')]:
    subset = test_df[test_df["region_id"] == rid]
    mae = mean_absolute_error(subset["y"], subset["y_pred"])
    rmse = np.sqrt(mean_squared_error(subset['y'], subset['y_pred']))
    print(f"{name} -> MAE : {mae:.2f} and RMSE : {rmse:.2f}")

Delhi -> MAE : 96.70 and RMSE : 189.24
Residential -> MAE : 1490.25 and RMSE : 1494.76


In [ ]:
summary = pd.DataFrame({
    "Model": [
        "Delhi-only (Model-A)",
        "Residential-only (Model-B)",
        "Global (Model-C)"
    ],
    "Generalization": [
        "Poor",
        "Poor",
        "Moderate"
    ],
    "Local Accuracy": [
        "Excellent",
        "Excellent",
        "Lower"
    ]
})

summary

* Specialised models achieve high local accuracy but perform poorly on non-local data (Poor Generalization).
* Global models perform moderately on unseen data.
* Energy-Demand forecasting is inherently domain-specific.

**Why global performs well on Delhi dataset?**
1. Dominates training set - Contributes more number of samples compared to residential.
2. Lower variance and smoother signals in city-level demand.
3. Weather features exist in Delhi but they are marked as 0.0 in Residential.
4. Residential demand is behavior-driven - Residential demand is behavior-driven.
5. Early tree splits are optimized for Delhi patterns - In LightGBM, early splits capture the strongest signals.